### Assignment 1: Implement a Bayesian Convolutional Neural Network

The Bayesian Convolutional Neural Network should distinguish between oncogenic and not oncogenic gene fusions exploiting the gene fusion sequence.

In details, analyze and explore the certainty classification value for the samples which are correctly classified.

In [4]:
to_drop_columns = ['Unnamed: 0', 'MainProteins', '5pEnsg', '3pEnsg', 'Version', '5pGeneFunctionality', '3pGeneFunctionality', '5pGeneDescription', '3pGeneDescription', '5pCommonName', '3pCommonName']

train_df = pd.read_table(TRAINING_SET).drop(to_drop_columns, 1)
test_df  = pd.read_table(TEST_SET)    .drop(to_drop_columns, 1)

train_df.head()

,FusionPair,Label,Chr5p,Coord5p,5pStrand,Chr3p,Coord3p,3pStrand,Proteins
0,CCDC6_RET,1,10,61665880,-,10,43612032,+,['MADSASESDTDGAGGNSSSSAAMQSSCSSTSGGGGGGGGGGGGG...
1,TMPRSS2_ERG,1,21,42870046,-,21,39795483,-,['MALNSELS']
2,TMPRSS2_ERG,1,21,42880008,-,21,39817544,-,['EALSVVSEDQSLFECAYGTPHLAKTEMTASSSSDYGQTSKMSPR...
3,TMPRSS2_ERG,1,21,42880008,-,21,39956869,-,['MIQTVPDPAAHIKEALSVVSEDQSLFECAYGTPHLAKTEMTASS...
4,TMPRSS2_ERG,1,21,42870046,-,21,39956869,-,['MALNSMIQTVPDPAAHIKEALSVVSEDQSLFECAYGTPHLAKTE...


In [5]:
train_df.Proteins = train_df.Proteins.apply(lambda protein: protein.replace("\'", "").replace("[", "").replace("]", "").replace(" ", ""))
test_df .Proteins = test_df .Proteins.apply(lambda protein: protein.replace("\'", "").replace("[", "").replace("]", "").replace(" ", ""))

train_df = clean_dataset(train_df)
test_df  = clean_dataset(test_df)
print("Cleaned protein sequences")

aminoacids_map = compute_one_hot_encoding_map(train_df)
print("Computed one hot encoding map")

train_df.Proteins = train_df.Proteins.apply(lambda protein: one_hot_encode(aminoacids_map, protein))
test_df .Proteins = test_df .Proteins.apply(lambda protein: one_hot_encode(aminoacids_map, protein))
print("Encoded proteins!")

Cleaned protein sequences
Computed one hot encoding map
Encoded proteins!


In [6]:
X_train = np.stack(train_df.Proteins)
X_test  = np.stack(test_df .Proteins)

combined = pd.concat([train_df.copy(), test_df.copy()])
labels   = utils.to_categorical(combined.Label)

y_train  = labels[:len(train_df)]
y_test   = labels[len(train_df):]

print("Features train dataset shape:\t" + str(X_train.shape))
print("Labels train dataset shape:\t"   + str(y_train.shape))

print("Features test dataset shape:\t"  + str(X_test.shape))
print("Labels test dataset shape:\t"    + str(y_test.shape))

Features train dataset shape:	(2118, 5000, 22)
Labels train dataset shape:	(2118, 2)
Features test dataset shape:	(896, 5000, 22)
Labels test dataset shape:	(896, 2)


#### Basic CNN 1D

In [9]:
EPOCHS      = 75
BATCH_SIZE  = 128
LR          = 0.001
INPUT_SHAPE = X_train[0].shape
NUM_CLASSES = len(np.unique(y_train))

cnn = CNN(
    EPOCHS,
    BATCH_SIZE,
    LR,
    INPUT_SHAPE,
    NUM_CLASSES,
    "Basic"
)

cnn.train_model(X_train, y_train)

Loaded Basic model.
Epoch 1/75
15/15 [==============================] - 2s 117ms/step - loss: 2.7707 - accuracy: 0.6156 - val_loss: 0.6145 - val_accuracy: 0.6967
Epoch 2/75
15/15 [==============================] - 1s 93ms/step - loss: 0.5054 - accuracy: 0.7819 - val_loss: 0.4334 - val_accuracy: 0.8294
Epoch 3/75
15/15 [==============================] - 1s 92ms/step - loss: 0.3872 - accuracy: 0.8464 - val_loss: 0.3253 - val_accuracy: 0.8863
Epoch 4/75
15/15 [==============================] - 1s 89ms/step - loss: 0.2992 - accuracy: 0.8862 - val_loss: 0.2636 - val_accuracy: 0.8957
Epoch 5/75
15/15 [==============================] - 1s 92ms/step - loss: 0.2277 - accuracy: 0.9046 - val_loss: 0.2166 - val_accuracy: 0.9336
Epoch 6/75
15/15 [==============================] - 1s 92ms/step - loss: 0.1736 - accuracy: 0.9350 - val_loss: 0.1983 - val_accuracy: 0.9289
Epoch 7/75
15/15 [==============================] - 1s 92ms/step - loss: 0.1363 - accuracy: 0.9544 - val_loss: 0.1887 - val_accuracy:

In [10]:
loss, acc   = cnn.model.evaluate(X_test, y_test, verbose=False);
predictions = cnn.model.predict(X_test)
print("Test done!")
print("Mean accuracy:\t {}\nLoss:\t\t {}".format(acc, loss))

Test done!
Mean accuracy:	 0.2276785671710968
Loss:		 4.505393981933594


#### Bayesian CNN 1D

In [7]:
EPOCHS      = 75
BATCH_SIZE  = 32
LR          = 0.0001
INPUT_SHAPE = X_train[0].shape
NUM_CLASSES = len(np.unique(y_train))
MODEL       = "Bayesian"

bcnn = CNN(
    EPOCHS,
    BATCH_SIZE,
    LR,
    INPUT_SHAPE,
    NUM_CLASSES, 
    MODEL
)

bcnn.train_model(X_train, y_train)

Loaded Bayesian model.
Epoch 1/75
60/60 [==============================] - 17s 280ms/step - loss: 1.9452 - accuracy: 0.5034 - val_loss: 0.8040 - val_accuracy: 0.4645
Epoch 2/75
60/60 [==============================] - 16s 274ms/step - loss: 0.7192 - accuracy: 0.5113 - val_loss: 0.7045 - val_accuracy: 0.4787
Epoch 3/75
60/60 [==============================] - 17s 278ms/step - loss: 0.6984 - accuracy: 0.5322 - val_loss: 0.7009 - val_accuracy: 0.5213
Epoch 4/75
60/60 [==============================] - 17s 285ms/step - loss: 0.6904 - accuracy: 0.5469 - val_loss: 0.6847 - val_accuracy: 0.5592
Epoch 5/75
60/60 [==============================] - 17s 291ms/step - loss: 0.6756 - accuracy: 0.5878 - val_loss: 0.6762 - val_accuracy: 0.5687
Epoch 6/75
60/60 [==============================] - 18s 302ms/step - loss: 0.6615 - accuracy: 0.6041 - val_loss: 0.6300 - val_accuracy: 0.6777
Epoch 7/75
60/60 [==============================] - 18s 306ms/step - loss: 0.6265 - accuracy: 0.6586 - val_loss: 0.5795

In [8]:
loss, acc   = bcnn.model.evaluate(X_test, y_test, verbose=False);
predictions = bcnn.model.predict(X_test)
print("Test done!")
print("Mean accuracy:\t {}\nLoss:\t\t {}".format(acc, loss))

predictions

Test done!
Mean accuracy:	 0.1819196492433548
Loss:		 1.2205983400344849


array([[0.7665872 , 0.23341279],
       [0.6576779 , 0.34232214],
       [0.8061638 , 0.19383624],
       ...,
       [0.6987473 , 0.30125272],
       [0.7235032 , 0.27649683],
       [0.73764235, 0.26235768]], dtype=float32)

#### Code

In [1]:
import numpy      as np
import pandas     as pd
import tensorflow as tf

from tensorflow              import keras, feature_column
from keras                   import utils
from keras.layers            import Input, Conv1D, Conv2D, Dropout, MaxPooling1D, MaxPooling2D, Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics         import accuracy_score

TEST_SET     = "test_set_1.csv"
TRAINING_SET = "training_set.csv"

In [2]:
class CNN:
    
    def __init__(self,
                 n_epochs, 
                 batch_size, 
                 learning_rate,
                 input_shape,
                 n_classes,
                 model,
                 lr_reduction_epoch=60,
                 mc_dropout_rate=0.6
                 ):

        self.n_epochs           = n_epochs
        self.batch_size         = batch_size
        self.learning_rate      = learning_rate
        self.input_shape        = input_shape
        self.n_classes          = n_classes
        self.rate               = mc_dropout_rate
        self.lr_reduction_epoch = lr_reduction_epoch

        if   model == "Basic":
          self.model            = self.BaseModel()
        elif model == "Bayesian":
          self.model            = self.BayesianModel()
        else:
          print("!!! Model not recognized !!!")

        print("Loaded " + model + " model.")
        
        return 

    def BaseModel(self):

      model = keras.models.Sequential()
      model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=self.input_shape))
      model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
      model.add(Dropout(0.5))
      model.add(MaxPooling1D(pool_size=2))
      model.add(Flatten())
      model.add(Dense(100, activation='relu'))
      model.add(Dense(self.n_classes, activation='softmax'))

      return model
      
    def BayesianModel(self):
      
      inputs = Input(shape=self.input_shape)

      # Block 1
      x = Conv1D(64, kernel_size = 3, padding = "same", activation = "relu", name = "block1_conv1")(inputs)
      x = Dropout(rate=self.rate)(x, training = True) 
      x = Conv1D(64, kernel_size = 3, padding = "same", activation = "relu", name = "block1_conv2")(x)
      x = Dropout(rate=self.rate)(x, training = True) 
      x = MaxPooling1D(2, strides=2, name = "block1_pool")(x)

      # Block 2
      x = Conv1D(128, kernel_size = 3, padding = "same", activation = "relu", name = "block2_conv1")(x)
      x = Dropout(rate=self.rate)(x, training = True) 
      x = Conv1D(128, kernel_size = 3, padding = "same", activation = "relu", name = "block2_conv2")(x)
      x = Dropout(rate=self.rate)(x, training = True) 
      x = MaxPooling1D(2, strides=2, name = "block2_pool")(x)

      # Block 3
      x = Conv1D(256, kernel_size = 3, padding = "same", activation = "relu", name = "block3_conv1")(x)
      x = Dropout(rate=self.rate)(x, training = True) 
      x = Conv1D(256, kernel_size = 3, padding = "same", activation = "relu", name = "block3_conv2")(x)
      x = Dropout(rate=self.rate)(x, training = True)
      x = Conv1D(256, kernel_size = 3, padding = "same", activation = "relu", name = "block3_conv3")(x)
      x = Dropout(rate=self.rate)(x, training = True)  
      x = MaxPooling1D(2, strides=2, name = "block3_pool")(x)

      # Block 4
      x = Conv1D(512, kernel_size = 3, padding = "same", activation = "relu", name = "block4_conv1")(x)
      x = Dropout(rate=self.rate)(x, training = True) 
      x = Conv1D(512, kernel_size = 3, padding = "same", activation = "relu", name = "block4_conv2")(x)
      x = Dropout(rate=self.rate)(x, training = True)
      x = Conv1D(512, kernel_size = 3, padding = "same", activation = "relu", name = "block4_conv3")(x)
      x = Dropout(rate=self.rate)(x, training = True)  
      x = MaxPooling1D(2, strides=2, name = "block4_pool")(x)

      # Block 5
      x = Conv1D(512, kernel_size = 3, padding = "same", activation = "relu", name = "block5_conv1")(x)
      x = Dropout(rate=self.rate)(x, training = True) 
      x = Conv1D(512, kernel_size = 3, padding = "same", activation = "relu", name = "block5_conv2")(x)
      x = Dropout(rate=self.rate)(x, training = True)
      x = Conv1D(512, kernel_size = 3, padding = "same", activation = "relu", name = "block5_conv3")(x)
      x = Dropout(rate=self.rate)(x, training = True)  
      x = MaxPooling1D(2, strides=2, name = "block5_pool")(x)

      # Classifier
      x = Flatten(name = "flatten")(x)
      x = Dense(4096, activation = "relu", name = "fc1")(x)
      x = Dropout(rate=self.rate)(x, training = True)
      x = Dense(4096, activation = "relu", name = "fc2")(x)
      x = Dropout(rate=self.rate)(x, training = True)
      x = Dense(self.n_classes, activation = "softmax", name = "predictions")(x)

      model = tf.keras.Model(inputs = inputs, outputs = x)

      return model
            
    def train_model(self, x_train, y_train):
        
        x_train, y_train, x_val, y_val = self._split_validation_data(x_train, y_train, 0.1)

        optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        
        self.model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        history = tf.keras.callbacks.History()
        scheduler_callback = tf.keras.callbacks.LearningRateScheduler(self.lr_scheduler, verbose=0)                                  

        self.model.fit(x=x_train, 
                       y=y_train, 
                       epochs=self.n_epochs,
                       batch_size=self.batch_size,
                       validation_data=(x_val, y_val),
                       callbacks=[
                          history,
                          scheduler_callback
                        ])   
        
    @staticmethod
    def _split_validation_data(x, y, validation_split):
        rand_indexes = np.random.permutation(x.shape[0])

        x = x[rand_indexes]
        y = y[rand_indexes]
        
        x_validation = x[:int(len(x) * validation_split)]
        y_validation = y[:int(len(x) * validation_split)]

        x_train = x[int(len(x) * validation_split):]
        y_train = y[int(len(x) * validation_split):]

        return x_train, y_train, x_validation, y_validation

    def lr_scheduler(self, epoch, lr):
        if epoch == self.lr_reduction_epoch:
            return lr * 0.1
        else:
            return lr 

In [3]:
def compute_one_hot_encoding_map(train_df):

  unique_aminoacids = set(list(''.join(train_df.Proteins.values))) # Set of aminoacids contained in the dataset
  unique_aminoacids.add("Filling")                                 # Additional fake aminoacid - used for filling the matrix, since every matrix should have the same dimension
  unique_aminoacids.add("Stop")                                    # Additional fake aminoacid - used as a stop row of the matrix

  aminoacids_number = len(unique_aminoacids)                       # Number of distinct aminoacids

  aminoacids_map = dict()                                          # Map initialization <aminoacid, one_hot_encoding>
  index          = 0
  zeros_list     = [0] * aminoacids_number                         # Reference zeros array. Iteratively filled with a 1 in the index-th index.

  for aminoacid in unique_aminoacids:

    encoding        = zeros_list.copy()                            # [0, 0, 0, 0, ..., 0]
    encoding[index] = 1                                            # [1, 0, 0, 0, ..., ...] (example)

    aminoacids_map[aminoacid] = encoding 

    index += 1

  return aminoacids_map                                            # <aminoacid, one_hot_encoding>

def one_hot_encode(aminoacids_map, protein, MAX_LEN = 5000):
  protein = list(protein)                                          # ["PROTEIN"] => ["P", "R", "O", "T", "E", "I", "N"]

  encoding = []                                                    # Matrix initialization

  for aminoacid in protein:
    encoding.append(aminoacids_map[aminoacid])                     # Mapping first N = len(protein) rows with the actual one hot encoding of the aminoacids

  for index in range(len(encoding), MAX_LEN - 1):                  # Filling additional MAX_LEN - N - 1 rows with a padding
    encoding.append(aminoacids_map["Filling"])

  encoding.append(aminoacids_map["Stop"])                          # Filling the last row with the stop encoding

  return np.array(encoding, dtype = np.float64) 

def clean_dataset(df):
  mask = df.Proteins.str.contains(",") == True

  new_rows = []

  for index, row in df[mask].iterrows():
    sequences = row.Proteins.split(",")
    
    for sequence in sequences:
      row.Proteins = sequence
      new_rows.append(row)

  df.drop(df[mask].index, inplace = True)
  return df.append(pd.DataFrame(new_rows, columns=df.columns)).reset_index()